In [2]:
import torch
from torch.autograd import Variable

In [3]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [4]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [6]:
print(x.grad_fn)
# y was created as a result of an operation, so it has a grad_fn.
print(y.grad_fn)

None


In [7]:
z = y*y*3
out = z.mean()

print(z, out)

(Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
, Variable containing:
 27
[torch.FloatTensor of size 1]
)


In [8]:
# Gradients
out.backward() 
# out.backward(torch.Tensor([1.0]))

In [9]:
# print gradient d(out)/dx
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



In [10]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x*2
print(y)

while y.data.norm() < 1000:
    y = y*2

print(y)

Variable containing:
 2.7245
 1.5470
 0.8247
[torch.FloatTensor of size 3]

Variable containing:
 1394.9673
  792.0643
  422.2717
[torch.FloatTensor of size 3]



In [11]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]

